In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa

In [2]:
# Es importante garantizar que los datos estén en el directorio correcto

DATA_DIR = Path.cwd().resolve().parent / "datos"
DATA_DIR

PosixPath('/Users/david.palacio/Documents/academia/data-projects-lab/projects/datos')

In [3]:
datos_titanic = pd.read_parquet(DATA_DIR / "01_datos_crudos_titanic.parquet")

In [4]:
# El primer argumento de la función "sample" es el número de filas que se desean seleccionar
# El segundo argumento es la semilla para el generador de números aleatorios (para hacer constantes los experimentos)
datos_titanic.sample(2, random_state=100)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
173,1,0,"Keeping, Mr. Edwin",male,32.5,0,0,113503,211.5,C132,C,?
843,3,0,"Hagland, Mr. Konrad Mathias Reiersen",male,?,1,0,65304,19.9667,?,S,?


In [5]:
# Para reemplazar valores nulos se puede usar el método "replace"

datos_titanic = datos_titanic.replace("?", np.nan)

datos_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pclass     1309 non-null   int64 
 1   survived   1309 non-null   int64 
 2   name       1309 non-null   object
 3   sex        1309 non-null   object
 4   age        1046 non-null   object
 5   sibsp      1309 non-null   int64 
 6   parch      1309 non-null   int64 
 7   ticket     1309 non-null   object
 8   fare       1308 non-null   object
 9   cabin      295 non-null    object
 10  embarked   1307 non-null   object
 11  home.dest  745 non-null    object
dtypes: int64(4), object(8)
memory usage: 122.8+ KB


Nótese que después de reemplazar los `?` con `nan` se tienen varios datos nulos en las variables:

* `age`
* `fare`
* `cabin`
* `embarked`
* `home.dest`

# Eliminación de columnas

* Aquellas columnas que tienen "muchos" valores nulos, es preferible removerlas para evitar introducir sesgos en los datos.
* La columa `ticket` es un identificador único por pasajero. Realmente no es muy valiosa a la hora de obtener información.

Para este caso de estudio, se eliminarán:
* `cabin`
* `ticket`
* `home.dest`

In [6]:
datos_titanic = datos_titanic.drop(columns=['cabin', 'home.dest', 'ticket'])

datos_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pclass    1309 non-null   int64 
 1   survived  1309 non-null   int64 
 2   name      1309 non-null   object
 3   sex       1309 non-null   object
 4   age       1046 non-null   object
 5   sibsp     1309 non-null   int64 
 6   parch     1309 non-null   int64 
 7   fare      1308 non-null   object
 8   embarked  1307 non-null   object
dtypes: int64(4), object(5)
memory usage: 92.2+ KB


# Variables Categóricas

## Ordinal

La variable categórica ordinal que se tiene en este conjunto de datos corresponde a `pclass`.

* `pclass`: una clasificación aproximada del estrato socio-económico de los pasajeros.
    * 1 = Upper
    * 2 = Middle
    * 3 = Lower

## Nominal

Las variables categóricas nominales que se tienen corresponden a `sex` y `embarked`.

* `sex`: género del pasajer@
    * female
    * male

* `embarked`: puerto de embarcación
    * C = Cherbourg
    * Q = Queenstown
    * S = Southampton

# Variables Numéricas

## Discretas

* `sibsp`: Este conjunto de datos tiene relaciones de familia.
    * Sibling = brother, sister, stepbrother, stepsister
    * Spouse = husband, wife (mistresses and fiancés were ignored)
    * `sibsp` = 0, 1, 2, 3, 4, 5, 8

* `parch`: The dataset defines family relations in this way…
    * Parent = mother, father
    * Child = daughter, son, stepdaughter, stepson
    * Some children travelled only with a nanny, therefore parch = 0 for them.
    * `parch` = 0, 1, 2, 3, 4, 5, 6

## Continuas

* `fare`: Tarifa del pasajero
* `age`: Edad del pasajero. Algunos valores están como decimales, por lo que se convertirán a enteros.

## Variables Booleanas

* `Survived`: 0 = No, 1 = Yes

## Variables Tipo String

* `name`: nombre del pasajero con el formato *apellido, titulo. Nombre*

# Conversión de datos

## Variables categóricas

In [7]:
cols_categoricas = ['pclass', 'sex', 'embarked']

datos_titanic[cols_categoricas] = datos_titanic[cols_categoricas].astype("category")

datos_titanic["pclass"] = pd.Categorical(
    datos_titanic["pclass"], categories=[3, 2, 1], ordered = True
)

## Variables numéricas

In [8]:
cols_numericas_flotantes = ['age', 'fare']

datos_titanic[cols_numericas_flotantes] = datos_titanic[cols_numericas_flotantes].astype("float64")

In [9]:
cols_numericas_enteros = ["sibsp", "parch"]

datos_titanic[cols_numericas_enteros] = datos_titanic[cols_numericas_enteros].astype("int8")

## Variables booleanas

In [10]:
cols_booleanas = ["survived"]

datos_titanic[cols_booleanas] = datos_titanic[cols_booleanas].astype("bool")

# Verificación final

In [11]:
datos_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   pclass    1309 non-null   category
 1   survived  1309 non-null   bool    
 2   name      1309 non-null   object  
 3   sex       1309 non-null   category
 4   age       1046 non-null   float64 
 5   sibsp     1309 non-null   int8    
 6   parch     1309 non-null   int8    
 7   fare      1308 non-null   float64 
 8   embarked  1307 non-null   category
dtypes: bool(1), category(3), float64(2), int8(2), object(1)
memory usage: 38.9+ KB


In [12]:
# Convierte un DataFrame de pandas en una tabla de PyArrow
# PyArrow es más eficiente en memoria que Pandas.

schema = pa.Table.from_pandas(datos_titanic).schema

# Guardar datos en formato `parquet`

In [13]:
# Obtener la ruta del notebook actual
ruta_datos_crudos = Path.cwd().resolve().parent / "datos"

ruta_archivo = ruta_datos_crudos / "02_datos_con_tipo_de_dato_ajustado_titanic.parquet"

# Crear directorio si no existe
ruta_datos_crudos.mkdir(parents=True, exist_ok=True)

# Guardar el DataFrame en el archivo parquet

# En este caso especificamos el esquema de datos. Útil para tener control total sobre los tipos de datos.
datos_titanic.to_parquet(ruta_archivo, index=False, schema=schema)